In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn


## Prepare data


In [24]:
# true_slope w: 2
# true_intercept b: 1

np.random.seed(42)

x = np.random.rand(100, 1)
noise = 0.1 * np.random.randn(100, 1)
y = x * 2 + 1 + noise

idx = np.arange(100)
np.random.shuffle(idx)

train_idx = idx[:80]
eval_idx = idx[80:]
x_train = x[train_idx]
y_train = y[train_idx]


In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"

x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

## initialize parameters

In [7]:
torch.manual_seed(42)

w = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
b = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)

print(b, w)

tensor([0.1288], requires_grad=True) tensor([0.3367], requires_grad=True)


In [11]:
# lr = 0.1
# epochs = 1000
# optimizer = optim.SGD([b, w], lr=lr) 

# lfn = nn.MSELoss(reduction='mean') 

# for _ in range(epochs):
#     y_hat = x_train_tensor * w + b
#     loss = lfn(y_hat, y_train_tensor)
#     loss.backward()

#     optimizer.step()
#     optimizer.zero_grad()

# print(b, w)

tensor([1.0235], requires_grad=True) tensor([1.9690], requires_grad=True)


## manual class model

In [26]:
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.b = nn.Parameter(torch.randn(1, requires_grad = True, dtype = torch.float))
        self.w = nn.Parameter(torch.randn(1, requires_grad = True, dtype = torch.float))

    def forward(self, x):
        return self.b + self.w * x

In [27]:
lr = 0.1
epochs = 1000
torch.manual_seed(42)

model = ManualLinearRegression().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr) 


lfn = nn.MSELoss(reduction='mean') 

for _ in range(epochs):
    model.train()
    y_hat = model(x_train_tensor)
    loss = lfn(y_hat, y_train_tensor)
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

print(model.state_dict())

OrderedDict({'b': tensor([1.0235]), 'w': tensor([1.9690])})
